# Imports

In [1]:
import spacy
import pandas as pd
from tqdm.auto import tqdm

import torch
torch.cuda.is_available()

True

# Create Docs

In [2]:
texts = pd.read_csv('../data/smart_source_texts.csv')['''Original Text'''].to_list()
termstrings = pd.read_csv('../data/smart_source_texts.csv')['E_KeyConcepts'].to_list()

In [3]:
gold_terms = [termstring.split(',') for termstring in termstrings]

# Extract Keywords

In [4]:
nlp = spacy.load('../training/model-last/')

In [5]:
doc_gen = tqdm(nlp.pipe(texts), total=len(texts))
pred_terms = [doc.ents for doc in doc_gen]

  0%|          | 0/11 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4256 > 4096). Running this sequence through the model will result in indexing errors


In [6]:
pred_terms

[(Osmosis,), (), (), (), (), (), (), (), (), (), ()]

In [17]:
terms_for_eval = []

for gold, pred, doc in zip(gold_terms, pred_terms, texts):
    terms_for_eval.append(([t for t in gold], [t.text for t in pred]))

In [18]:
terms_for_eval

[(['passive transport',
   'transport',
   'diffusion',
   'cell',
   'plasma membrane',
   'concentration',
   'substance',
   'osmosis'],
  []),
 (['autotroph',
   'calvin cycle',
   'light-dependent reaction',
   'chloroplast',
   'chemical energy',
   'photosynthesis',
   'chlorophyll',
   'solar energy',
   'mesophyll'],
  []),
 (['chloroplast',
   'calvin cycle',
   'light-dependent reaction',
   'photosynthesis',
   'carbon dioxide',
   'oxygen',
   'thylakoid',
   'autotroph',
   'heterotroph',
   'glucose'],
  []),
 (['dominant allele',
   'heterozygote',
   'allele',
   'law',
   'mendel',
   'phenotype',
   'genotype',
   'homozygous recessive',
   'heterozygote genotype',
   'other allele'],
  []),
 (['rna polymerase',
   'gene',
   'elongation',
   'mrna transcript',
   'initiation',
   'dna',
   'termination',
   'transcription',
   'promoter sequence'],
  []),
 (['termination',
   'translation',
   'mrna',
   'amino acid',
   'initiation',
   'elongation',
   'protein',


In [62]:
f = open('results/macroeconomics-2e-results.txt','w', encoding='utf-8')

for name, gold, pred in terms_for_eval:
    f.write(name + '\n\n')
    i = 0
    j = 0
    while i < len(gold) and j < len(pred):
        if gold[i] in pred:
            if gold[i] == pred[j]:
                s = f'Gold: {gold[i]: <35} Pred: {pred[j]}'
                i += 1
                j +=1
            else:
                s = ' '*42 + f'Pred: {pred[j]}'
                j += 1
        elif i < len(gold):
            s = f'Gold: {gold[i]: <35}'
            i += 1
        else:
            s = ' '*42 + f'Pred: {pred[j]}'
            j += 1
        f.write(s + '\n')
    f.write('\n')
f.close()

# Send terms to csv

In [ ]:
terms_for_eval

In [12]:
import pandas as pd
pd.DataFrame(terms_for_eval, columns=['Section', 'Gold', 'Pred']).to_csv('results/macroeconomics-2e-results.csv', index=False)